In [ ]:
!pip3 install pytelegrambotapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 8.8 MB/s eta 0:00:00


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 5.6 MB/s eta 0:00:00


In [ ]:
!pip install numpy_financial

In [ ]:
# !git clone https://github.com/larichevavd/data.git
# df = pd.read_csv('/content/data/sell_bot.csv')
# rent = pd.read_csv('/content/data/rent_final.csv')
# df

In [ ]:
import pandas as pd
import telebot
from telebot import types
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import numpy_financial as npf

TOKEN = '...'
bot = telebot.TeleBot(TOKEN)

try:
    df = pd.read_csv('sell_bot.csv')
    rent = pd.read_csv('rent_final.csv')
except:
    !git clone https://github.com/larichevavd/data.git
    df = pd.read_csv('/content/data/sell_bot.csv')
    rent = pd.read_csv('/content/data/rent_final.csv')

cat_cols = ['Город', 'Район', 'Метро']
rent_clean = rent.drop(columns=['url'])
y = rent_clean['За метр']
X = rent_clean[['Город', 'Этаж', 'Этажность', 'Количество комнат', 'Площадь', 'Район', 'Метро']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1107)
model = CatBoostRegressor(
    iterations=1200,
    learning_rate=0.2,
    depth=6,
    l2_leaf_reg=3,
    verbose=0,
    cat_features=[0, 5, 6]
)
model.fit(X_train, y_train)

user_states = {}

def reset_state(chat_id):
    user_states[chat_id] = {'state': 'function_selection'}

@bot.message_handler(commands=['start'])
def send_welcome(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Подобрать наиболее доходный объект")
    btn2 = types.KeyboardButton("Рассчитать доходность своего объекта")
    markup.add(btn1, btn2)

    welcome_text = """
    Привет! Я бот для поддержки принятия решений при инвестировании в недвижимость. Я могу:
    1. Подобрать наиболее доходный объект из базы данных
    2. Рассчитать прогнозную доходность для вашего объекта

    Выберите одну из функций:
    """
    bot.send_message(message.chat.id, welcome_text, reply_markup=markup)
    reset_state(message.chat.id)

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'function_selection')
def handle_function_selection(message):
    if message.text == "Подобрать наиболее доходный объект":
        start_function_1(message.chat.id)
    elif message.text == "Рассчитать доходность своего объекта":
        start_function_2(message.chat.id)
    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        btn1 = types.KeyboardButton("Подобрать наиболее доходный объект")
        btn2 = types.KeyboardButton("Рассчитать доходность своего объекта")
        markup.add(btn1, btn2)
        bot.send_message(message.chat.id, "Пожалуйста, выберите одну из кнопок:", reply_markup=markup)


# ФУНКЦИЯ 1

def start_function_1(chat_id):
    user_states[chat_id] = {'state': 'filter_city', 'filters': {}, 'results': None, 'current_index': 0}
    ask_city(chat_id)

def ask_city(chat_id):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=3)
    cities = ["Москва", "Санкт-Петербург", "Новосибирск", "Екатеринбург", "Казань",
              "Красноярск", "Нижний Новгород", "Челябинск", "Уфа", "Самара",
              "Ростов-на-Дону", "Краснодар", "Омск", "Воронеж", "Пермь", "Волгоград", "Любой"]
    buttons = [types.KeyboardButton(city) for city in cities]
    markup.add(*buttons)
    bot.send_message(chat_id, "Выберите город:", reply_markup=markup)

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'filter_city')
def handle_city(message):
    text = message.text.lower()
    if text.lower() == 'любой':
        user_states[message.chat.id]['filters']['city'] = None
    else:
        user_states[message.chat.id]['filters']['city'] = message.text

    user_states[message.chat.id]['state'] = 'filter_district'
    ask_district(message.chat.id)

def ask_district(chat_id):
    city = user_states[chat_id]['filters'].get('city')
    if city is None:
        user_states[chat_id]['state'] = 'filter_floor'
        ask_floor(chat_id)
        return

    districts = df[(df['Город'] == city) &
                  (df['Район'].notna()) &
                  (df['Район'] != 'Отсутствует') &
                  (df['Район'] != city) &
                  (df['Район'].str.len() <= 30)]['Район'].unique()

    if len(districts) == 0:
        bot.send_message(chat_id, "Для выбранного города нет данных по районам.")
        user_states[chat_id]['state'] = 'filter_floor'
        ask_floor(chat_id)
        return

    districts_list = sorted(districts)
    districts_text = "Выберите район (введите число):\n"
    for i, district in enumerate(districts_list, 1):
        districts_text += f"{i}. {district}\n"
    districts_text += "\n0. Любой район"

    user_states[chat_id]['districts_list'] = districts_list
    bot.send_message(chat_id, districts_text, reply_markup=types.ReplyKeyboardRemove())

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'filter_district')
def handle_district(message):
    try:
        choice = int(message.text)
        districts_list = user_states[message.chat.id]['districts_list']

        if choice == 0:
            user_states[message.chat.id]['filters']['district'] = None
        elif 1 <= choice <= len(districts_list):
            user_states[message.chat.id]['filters']['district'] = districts_list[choice-1]
        else:
            raise ValueError

        user_states[message.chat.id]['state'] = 'filter_metro'
        ask_metro(message.chat.id)
    except:
        bot.send_message(message.chat.id, "Пожалуйста, введите число из списка.")

def ask_metro(chat_id):
    city = user_states[chat_id]['filters'].get('city')
    if city is None or city not in ['Москва', 'Санкт-Петербург', 'Казань', 'Нижний Новгород', 'Екатеринбург', 'Новосибирск', 'Самара']:
        user_states[chat_id]['state'] = 'filter_floor'
        ask_floor(chat_id)
        return

    metro_stations = df[(df['Город'] == city) &
                       (df['Метро'].notna()) &
                       (df['Метро'] != 'Отсутствует')]['Метро'].unique()

    if len(metro_stations) == 0:
        user_states[chat_id]['state'] = 'filter_floor'
        ask_floor(chat_id)
        return

    metro_list = sorted(metro_stations)
    metro_text = "Выберите станцию метро (введите число):\n"
    for i, station in enumerate(metro_list, 1):
        metro_text += f"{i}. {station}\n"
    metro_text += "\n0. Любая станция"

    user_states[chat_id]['metro_list'] = metro_list
    bot.send_message(chat_id, metro_text)

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'filter_metro')
def handle_metro(message):
    try:
        choice = int(message.text)
        metro_list = user_states[message.chat.id]['metro_list']

        if choice == 0:
            user_states[message.chat.id]['filters']['metro'] = None
        elif 1 <= choice <= len(metro_list):
            user_states[message.chat.id]['filters']['metro'] = metro_list[choice-1]
        else:
            raise ValueError

        user_states[message.chat.id]['state'] = 'filter_floor'
        ask_floor(message.chat.id)
    except:
        bot.send_message(message.chat.id, "Пожалуйста, введите число из списка.")

def ask_floor(chat_id):
    instructions = """
    Укажите этаж:
    - Введите число для конкретного этажа
    - "кроме первого"
    - "кроме последнего"
    - "кроме первого и последнего"
    - "выше {число}" (например, "выше 5")
    - "ниже {число}" (например, "ниже 10")
    - "любой"
    """
    bot.send_message(chat_id, instructions)

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'filter_floor')
def handle_floor(message):
    text = message.text.lower()
    if text.lower() == 'любой':
        user_states[message.chat.id]['filters']['floor'] = None
    else:
        user_states[message.chat.id]['filters']['floor'] = text

    user_states[message.chat.id]['state'] = 'filter_floors_total'
    ask_floors_total(message.chat.id)

def ask_floors_total(chat_id):
    instructions = """
    Укажите этажность дома:
    - Введите число для конкретной этажности
    - "выше {число}" (например, "выше 5")
    - "ниже {число}" (например, "ниже 10")
    - "любая"
    """
    bot.send_message(chat_id, instructions)

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'filter_floors_total')
def handle_floors_total(message):
    text = message.text.lower()
    if text == 'любая':
        user_states[message.chat.id]['filters']['floors_total'] = None
    else:
        user_states[message.chat.id]['filters']['floors_total'] = text

    user_states[message.chat.id]['state'] = 'filter_rooms'
    ask_rooms(message.chat.id)

def ask_rooms(chat_id):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = [
        types.KeyboardButton("Студия"),
        types.KeyboardButton("1"),
        types.KeyboardButton("2"),
        types.KeyboardButton("3"),
        types.KeyboardButton("Любое")
    ]
    markup.add(*buttons)
    bot.send_message(chat_id, "Выберите количество комнат:", reply_markup=markup)

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'filter_rooms')
def handle_rooms(message):
    text = message.text.lower()
    if text == 'любое' or text == 'Любое':
        user_states[message.chat.id]['filters']['rooms'] = None
    elif text == 'студия':
        user_states[message.chat.id]['filters']['rooms'] = 0
    else:
        try:
            user_states[message.chat.id]['filters']['rooms'] = int(text)
        except ValueError:
            bot.send_message(message.chat.id, "Пожалуйста, выберите один из вариантов.")
            return

    user_states[message.chat.id]['state'] = 'filter_area'
    ask_area(message.chat.id)

def ask_area(chat_id):
    instructions = """
    Укажите площадь (кв.м):
    - Введите два числа через дефис (например, "30-50")
    - "любое"
    """
    bot.send_message(chat_id, instructions, reply_markup=types.ReplyKeyboardRemove())

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'filter_area')
def handle_area(message):
    text = message.text.lower()
    if text == 'любое' or text == 'Любое':
        user_states[message.chat.id]['filters']['area'] = None
    else:
        try:
            area_from, area_to = map(float, text.split('-'))
            user_states[message.chat.id]['filters']['area'] = (area_from, area_to)
        except:
            bot.send_message(message.chat.id, "Пожалуйста, введите два числа через дефис или 'любое'.")
            return

    user_states[message.chat.id]['state'] = 'filter_price'
    ask_price(message.chat.id)

def ask_price(chat_id):
    instructions = """
    Укажите цену (руб):
    - Введите два числа через дефис (например, "5000000-10000000")
    - "любая"
    """
    bot.send_message(chat_id, instructions)

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'filter_price')
def handle_price(message):
    text = message.text.lower()
    if text == 'любая':
        user_states[message.chat.id]['filters']['price'] = None
    else:
        try:
            price_from, price_to = map(float, text.split('-'))
            user_states[message.chat.id]['filters']['price'] = (price_from, price_to)
        except:
            bot.send_message(message.chat.id, "Пожалуйста, введите два числа через дефис или 'любая'.")
            return

    user_states[message.chat.id]['state'] = 'searching'
    find_best_property(message.chat.id)

def apply_floor_filter(data, floor_filter):
    if floor_filter is None:
        return data

    if isinstance(floor_filter, int):
        return data[data['Этаж'] == floor_filter]

    if floor_filter == 'кроме первого':
        return data[data['Этаж'] != 1]
    elif floor_filter == 'кроме последнего':
        return data[data['Этаж'] != data['Этажность']]
    elif floor_filter == 'кроме первого и последнего':
        return data[(data['Этаж'] != 1) & (data['Этаж'] != data['Этажность'])]
    elif floor_filter.startswith('выше '):
        try:
            num = int(floor_filter.split()[1])
            return data[data['Этаж'] > num]
        except:
            return data
    elif floor_filter.startswith('ниже '):
        try:
            num = int(floor_filter.split()[1])
            return data[data['Этаж'] < num]
        except:
            return data

    return data

def apply_floors_total_filter(data, floors_total_filter):
    if floors_total_filter is None:
        return data

    if isinstance(floors_total_filter, int):
        return data[data['Этажность'] == floors_total_filter]

    if floors_total_filter.startswith('выше '):
        try:
            num = int(floors_total_filter.split()[1])
            return data[data['Этажность'] > num]
        except:
            return data
    elif floors_total_filter.startswith('ниже '):
        try:
            num = int(floors_total_filter.split()[1])
            return data[data['Этажность'] < num]
        except:
            return data

    return data

def find_best_property(chat_id):
    filters = user_states[chat_id]['filters']
    filtered_data = df.copy()

    if filters.get('city') is not None:
        filtered_data = filtered_data[filtered_data['Город'] == filters['city']]

    if filters.get('district') is not None:
        filtered_data = filtered_data[filtered_data['Район'] == filters['district']]

    if filters.get('metro') is not None:
        filtered_data = filtered_data[filtered_data['Метро'] == filters['metro']]

    if filters.get('floor') is not None:
        filtered_data = apply_floor_filter(filtered_data, filters['floor'])

    if filters.get('floors_total') is not None:
        filtered_data = apply_floors_total_filter(filtered_data, filters['floors_total'])

    if filters.get('rooms') is not None:
        filtered_data = filtered_data[filtered_data['Количество комнат'] == filters['rooms']]

    if filters.get('area') is not None:
        area_from, area_to = filters['area']
        filtered_data = filtered_data[(filtered_data['Площадь'] >= area_from) &
                                     (filtered_data['Площадь'] <= area_to)]

    if filters.get('price') is not None:
        price_from, price_to = filters['price']
        filtered_data = filtered_data[(filtered_data['Цена'] >= price_from) &
                                     (filtered_data['Цена'] <= price_to)]

    if len(filtered_data) > 0:
        filtered_data = filtered_data.sort_values('Доходность', ascending=False)
        user_states[chat_id]['results'] = filtered_data
        user_states[chat_id]['current_index'] = 0
        show_property(chat_id, 0)
    else:
        response = "Не найдено объектов по вашим критериям."
        bot.send_message(chat_id, response)
        offer_restart(chat_id)

def show_property(chat_id, index):
    results = user_states[chat_id]['results']
    if index >= len(results):
        bot.send_message(chat_id, "Больше нет подходящих объектов.")
        offer_restart(chat_id)
        return

    property = results.iloc[index]

    input_data = pd.DataFrame({
        'Город': [property['Город']],
        'Этаж': [property['Этаж']],
        'Этажность': [property['Этажность']],
        'Количество комнат': [property['Количество комнат']],
        'Площадь': [property['Площадь']],
        'Район': [property['Район']],
        'Метро': [property.get('Метро', 'Отсутствует')]
    })

    try:
        rent_per_meter = model.predict(input_data)[0]
        rent_full = rent_per_meter * property['Площадь']
    except:
        rent_full = property.get('Аренда', 'неизвестно')

    response = f"""
    Найден объект {index+1} из {len(results)}:

    Город: {property['Город']}
    Район: {property['Район']}
    Метро: {property.get('Метро', 'не указано')}
    Этаж: {property['Этаж']} из {property['Этажность']}
    Комнат: {'студия' if property['Количество комнат'] == 0 else property['Количество комнат']}
    Площадь: {property['Площадь']} кв.м
    Цена: {property['Цена']:,.0f} руб
    Прогнозируемая аренда: {rent_full:,.0f} руб/мес
    Прогнозируемая доходность: {100*property['Доходность']:.2f}%
    Ссылка: {property['url']}
    """

    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Другой вариант")
    btn2 = types.KeyboardButton("Стоп")
    markup.add(btn1, btn2)

    bot.send_message(chat_id, response, reply_markup=markup)
    user_states[chat_id]['state'] = 'showing_results'

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'showing_results')
def handle_results_choice(message):
    if message.text == "Другой вариант":
        user_states[message.chat.id]['current_index'] += 1
        show_property(message.chat.id, user_states[message.chat.id]['current_index'])
    elif message.text == "Стоп":
        ask_mortgage_option(message.chat.id)
    else:
        bot.send_message(message.chat.id, "Пожалуйста, выберите один из вариантов.")

def ask_mortgage_option(chat_id):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Да, проверить ипотеку")
    btn2 = types.KeyboardButton("Нет, завершить")
    markup.add(btn1, btn2)

    text = """
    Если Вы попадаете под условия льготных ипотечных программ, \
то стоит проверить, можете ли Вы получить больший доход, если не сразу купите квартиру, \
а возьмете ипотеку по низкой ставке и положите остальные деньги в банк на накопительный счет.

    Хотите проверить вариант с ипотекой?
    """

    bot.send_message(chat_id, text, reply_markup=markup)
    user_states[chat_id]['state'] = 'mortgage_choice'

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'mortgage_choice')
def handle_mortgage_choice(message):
    if message.text == "Да, проверить ипотеку":
        bot.send_message(message.chat.id, "Введите ставку по ипотеке (% годовых):", reply_markup=types.ReplyKeyboardRemove())
        user_states[message.chat.id]['state'] = 'mortgage_rate'
    elif message.text == "Нет, завершить":
        offer_restart(message.chat.id)
    else:
        bot.send_message(message.chat.id, "Пожалуйста, выберите один из вариантов.")

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'mortgage_rate')
def handle_mortgage_rate(message):
    try:
        mortgage_rate = float(message.text)
        user_states[message.chat.id]['mortgage_rate'] = mortgage_rate
        bot.send_message(message.chat.id, "Введите ставку по вкладу (% годовых):")
        user_states[message.chat.id]['state'] = 'deposit_rate'
    except:
        bot.send_message(message.chat.id, "Пожалуйста, введите число.")

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'deposit_rate')
def handle_deposit_rate(message):
    try:
        deposit_rate = float(message.text)
        user_states[message.chat.id]['deposit_rate'] = deposit_rate
        calculate_mortgage_option(message.chat.id)
    except:
        bot.send_message(message.chat.id, "Пожалуйста, введите число.")

def calculate_mortgage_option(chat_id):
    if 'current_property' in user_states[chat_id]:
        price = user_states[chat_id]['current_property']['price']
        rent_full = user_states[chat_id]['current_property']['rent']
    else:
        current_index = user_states[chat_id]['current_index']
        property = user_states[chat_id]['results'].iloc[current_index]
        price = property['Цена']
        rent_full = property.get('Аренда', 0)

    mortgage_rate = user_states[chat_id]['mortgage_rate'] / 100 / 12
    deposit_rate = user_states[chat_id]['deposit_rate'] / 100
    result = False
    for downpayment_percent in [0.8, 0.9]:
        loan_amount = price * (1 - downpayment_percent)
        n_payments = 5 * 12

        try:
            monthly_payment = npf.pmt(mortgage_rate, n_payments, -loan_amount)
        except:
            monthly_payment = 0

        if monthly_payment > rent_full:
            response = f"""
        При первоначальном взносе {int(downpayment_percent*100)}%:\n
    - Месячный платеж: {monthly_payment:,.0f} руб
    - Прогнозируемая аренда: {rent_full:,.0f} руб\n
    Этот вариант не подходит, так как платеж превышает возможную аренду.
            """
            bot.send_message(chat_id, response)
            continue


        deposit_amount = price * (1 - downpayment_percent)
        current_rate = deposit_rate

        for year in range(1, 6):  # 5 лет
            deposit_amount = deposit_amount * (1 + current_rate)
            current_rate = min(6, current_rate - 0.025)

        total_payments = monthly_payment * 12 * 5

        difference = deposit_amount - total_payments
        if difference > 100000:
            result =  'Можно взять ипотеку, так как доход от вклада больше, чем проценты по ипотеке'
        elif difference > 0:
            result = 'Разница небольшая'
        else:
            result = 'Не стоит брать ипотеку, так как проценты по ипотеке больше, чем доход от вклада'

        response = f"""
      При первоначальном взносе {int(downpayment_percent*100)}%:\n
    - Месячный платеж: {monthly_payment:,.0f} руб
    - Прогнозируемая аренда: {rent_full:,.0f} руб
    - Доход от вклада за 5 лет: {deposit_amount:,.0f} руб
    - Платежи по ипотеке за 5 лет: {total_payments:,.0f} руб
    - Чистая выгода: {difference:,.0f} руб\n
    - Вывод: {result}
        """

        bot.send_message(chat_id, response)

    offer_restart(chat_id)

# ФУНКЦИЯ 2

def start_function_2(chat_id):
    if model is None:
        bot.send_message(chat_id, "Извините, функция расчета доходности временно недоступна.")
        offer_restart(chat_id)
        return

    user_states[chat_id] = {'state': 'calc_city', 'params': {}}
    ask_calc_city(chat_id)

def ask_calc_city(chat_id):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=3)
    cities = ["Москва", "Санкт-Петербург", "Новосибирск", "Екатеринбург", "Казань",
              "Красноярск", "Нижний Новгород", "Челябинск", "Уфа", "Самара",
              "Ростов-на-Дону", "Краснодар", "Омск", "Воронеж", "Пермь", "Волгоград"]
    buttons = [types.KeyboardButton(city) for city in cities]
    markup.add(*buttons)
    bot.send_message(chat_id, "Укажите город объекта:", reply_markup=markup)

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'calc_city')
def handle_calc_city(message):
    user_states[message.chat.id]['params']['city'] = message.text
    user_states[message.chat.id]['state'] = 'calc_district'
    ask_calc_district(message.chat.id)

def ask_calc_district(chat_id):
    city = user_states[chat_id]['params'].get('city')
    districts = df[(df['Город'] == city) &
                  (df['Район'].notna()) &
                  (df['Район'] != 'Отсутствует') &
                  (df['Район'] != city) &
                  (df['Район'].str.len() <= 30)]['Район'].unique()

    if len(districts) == 0:
        user_states[chat_id]['params']['district'] = 'Отсутствует'
        user_states[chat_id]['state'] = 'calc_metro'
        ask_calc_metro(chat_id)
        return

    districts_list = sorted(districts)
    districts_text = "Укажите район (введите число):\n"
    for i, district in enumerate(districts_list, 1):
        districts_text += f"{i}. {district}\n"
    districts_text += f"{len(districts_list)+1}. Другой"

    user_states[chat_id]['districts_list'] = districts_list
    bot.send_message(chat_id, districts_text, reply_markup=types.ReplyKeyboardRemove())

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'calc_district')
def handle_calc_district(message):
    try:
        districts_list = user_states[message.chat.id]['districts_list']
        choice = int(message.text)

        if 1 <= choice <= len(districts_list):
            user_states[message.chat.id]['params']['district'] = districts_list[choice-1]
        elif choice == len(districts_list)+1:
            user_states[message.chat.id]['params']['district'] = 'Отсутствует'
        else:
            raise ValueError

        user_states[message.chat.id]['state'] = 'calc_metro'
        ask_calc_metro(message.chat.id)
    except:
        bot.send_message(message.chat.id, "Пожалуйста, введите число из списка.")

def ask_calc_metro(chat_id):
    city = user_states[chat_id]['params'].get('city')
    if city not in ['Москва', 'Санкт-Петербург', 'Казань', 'Нижний Новгород', 'Екатеринбург', 'Новосибирск', 'Самара']:
        user_states[chat_id]['params']['metro'] = 'Отсутствует'
        user_states[chat_id]['state'] = 'calc_floor'
        ask_calc_floor(chat_id)
        return

    metro_stations = df[(df['Город'] == city) &
                       (df['Метро'].notna()) &
                       (df['Метро'] != 'Отсутствует')]['Метро'].unique()

    if len(metro_stations) == 0:
        user_states[chat_id]['params']['metro'] = 'Отсутствует'
        user_states[chat_id]['state'] = 'calc_floor'
        ask_calc_floor(chat_id)
        return

    metro_list = sorted(metro_stations)
    metro_text = "Укажите станцию метро (введите число):\n"
    for i, station in enumerate(metro_list, 1):
        metro_text += f"{i}. {station}\n"
    metro_text += f"{len(metro_list)+1}. Другая"

    user_states[chat_id]['metro_list'] = metro_list
    bot.send_message(chat_id, metro_text)

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'calc_metro')
def handle_calc_metro(message):
    try:
        metro_list = user_states[message.chat.id]['metro_list']
        choice = int(message.text)

        if 1 <= choice <= len(metro_list):
            user_states[message.chat.id]['params']['metro'] = metro_list[choice-1]
        elif choice == len(metro_list)+1:
            user_states[message.chat.id]['params']['metro'] = 'Отсутствует'
        else:
            raise ValueError

        user_states[message.chat.id]['state'] = 'calc_floor'
        ask_calc_floor(message.chat.id)
    except:
        bot.send_message(message.chat.id, "Пожалуйста, введите число из списка.")

def ask_calc_floor(chat_id):
    bot.send_message(chat_id, "Укажите этаж объекта (число):")

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'calc_floor')
def handle_calc_floor(message):
    try:
        floor = int(message.text)
        user_states[message.chat.id]['params']['floor'] = floor
        user_states[message.chat.id]['state'] = 'calc_floors_total'
        ask_calc_floors_total(message.chat.id)
    except:
        bot.send_message(message.chat.id, "Пожалуйста, введите число.")

def ask_calc_floors_total(chat_id):
    bot.send_message(chat_id, "Укажите этажность дома (число):")

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'calc_floors_total')
def handle_calc_floors_total(message):
    try:
        floors_total = int(message.text)
        user_states[message.chat.id]['params']['floors_total'] = floors_total
        user_states[message.chat.id]['state'] = 'calc_rooms'
        ask_calc_rooms(message.chat.id)
    except:
        bot.send_message(message.chat.id, "Пожалуйста, введите число.")

def ask_calc_rooms(chat_id):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = [
        types.KeyboardButton("Студия"),
        types.KeyboardButton("1"),
        types.KeyboardButton("2"),
        types.KeyboardButton("3")
    ]
    markup.add(*buttons)
    bot.send_message(chat_id, "Укажите количество комнат:", reply_markup=markup)

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'calc_rooms')
def handle_calc_rooms(message):
    text = message.text.lower()
    if text == 'студия':
        user_states[message.chat.id]['params']['rooms'] = 0
    else:
        try:
            rooms = int(text)
            if rooms not in [1, 2, 3]:
                raise ValueError
            user_states[message.chat.id]['params']['rooms'] = rooms
        except:
            bot.send_message(message.chat.id, "Пожалуйста, выберите один из вариантов.")
            return

    user_states[message.chat.id]['state'] = 'calc_area'
    ask_calc_area(message.chat.id)

def ask_calc_area(chat_id):
    bot.send_message(chat_id, "Укажите площадь объекта (кв.м, число):", reply_markup=types.ReplyKeyboardRemove())

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'calc_area')
def handle_calc_area(message):
    try:
        area = float(message.text)
        user_states[message.chat.id]['params']['area'] = area
        user_states[message.chat.id]['state'] = 'calc_price'
        ask_calc_price(message.chat.id)
    except:
        bot.send_message(message.chat.id, "Пожалуйста, введите число.")

def ask_calc_price(chat_id):
    bot.send_message(chat_id, "Укажите стоимость объекта (руб, число):")

@bot.message_handler(func=lambda message: user_states.get(message.chat.id, {}).get('state') == 'calc_price')
def handle_calc_price(message):
    try:
        price = float(message.text)
        user_states[message.chat.id]['params']['price'] = price

        calculate_profitability(message.chat.id)
    except:
        bot.send_message(message.chat.id, "Пожалуйста, введите число.")

def calculate_profitability(chat_id):
    params = user_states[chat_id]['params']

    input_data = pd.DataFrame({
        'Город': [params['city']],
        'Этаж': [params['floor']],
        'Этажность': [params['floors_total']],
        'Количество комнат': [params['rooms']],
        'Площадь': [params['area']],
        'Район': [params['district']],
        'Метро': [params['metro']]
    })

    try:
        rent_per_meter = model.predict(input_data)[0]

        rent_full = rent_per_meter * params['area']
        SA = rent_full
        SP = params['price']

        user_states[chat_id]['current_property'] = {
            'price': SP,
            'rent': rent_full
        }

        profitability = ((SA * 0.87 * 12) - (SP * 0.8 * 0.001) - (SA * 0.05 * 12)) / SP
        profitability_percent = profitability * 100

        response = f"""
    Результаты расчета:

    Прогнозируемая аренда: {rent_full:,.0f} руб/мес
    Стоимость объекта: {SP:,.0f} руб
    Прогнозируемая доходность: {profitability_percent:.1f}% годовых
        """
    except Exception as e:
        print(f"Ошибка при расчете: {e}")
        response = "Произошла ошибка при расчете доходности. Пожалуйста, попробуйте позже."

    bot.send_message(chat_id, response)
    ask_mortgage_option(chat_id)

# ОБЩИЕ ФУНКЦИИ

def offer_restart(chat_id):
    reset_state(chat_id)
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("/start")
    markup.add(btn1)
    bot.send_message(chat_id,
        "Операция завершена. Для нового запроса нажмите /start",
        reply_markup=markup)

@bot.message_handler(func=lambda message: True)
def handle_unrecognized(message):
    current_state = user_states.get(message.chat.id, {}).get('state')

    if current_state is None or current_state == 'function_selection':
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        btn1 = types.KeyboardButton("Подобрать наиболее доходный объект")
        btn2 = types.KeyboardButton("Рассчитать доходность своего объекта")
        markup.add(btn1, btn2)
        bot.send_message(message.chat.id, "Пожалуйста, выберите одну из предложенных функций:", reply_markup=markup)
    else:
        bot.send_message(message.chat.id, "Пожалуйста, завершите текущую операцию или отправьте /start для перезапуска")

print("Бот запущен...")
bot.infinity_polling()

Cloning into 'data'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), 5.49 MiB | 4.12 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Бот запущен...
